In [1]:
import re
import jieba
import jieba.analyse
import json
import random
import operator
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [2]:
from elasticsearch import Elasticsearch
import elasticsearch.helpers
from datetime import datetime

In [3]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [4]:
# Find top10
res = es.search(index='facebook', doc_type='user', body= {'query': {'match': {'gender': 'female'}}})
print("%d documents found" % res['hits']['total'])

58 documents found


In [5]:
# Take all result
users = list(elasticsearch.helpers.scan(es, index='facebook', doc_type='user'))
total_users = [user['_id'] for user in users]
total_user_with_gender = [(user['_id'], user['_source'].get('gender')) for user in users]
print(len(total_user_with_gender))

143


In [6]:
no_message = ['1618047998214350', '1624029027647185', '1454298984658260', '1805190672831590']
foreigners = ['1695358370488750', '10213401737335928', '1532260126834371', '10154597524691479', '10155404471078548', '10158882655745234']

In [7]:
total_users = [user for user in total_users if user not in (no_message + foreigners)]
print(len(total_users))

133


In [8]:
user_dict = {'male': [], 'female': []}
for user in total_user_with_gender:
    if user[0] not in (no_message + foreigners):
        if user[1] == 'male':
            user_dict['male'].append(user[0])
        else:
            user_dict['female'].append(user[0])

In [9]:
print('male: ' + str(len(user_dict['male'])) + '\nfemale: ' + str(len(user_dict['female'])))

male: 79
female: 54


In [10]:
# Filtering urls
def filtering_url(messages):
    filtered_messages = [re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', message.replace('\n', '').replace('\r', '')) for message in messages]
    return filtered_messages

In [11]:
# Mapping user posts
def get_posts(users):
    posts_dic = {}
    for user in users:
        # print(user)
        posts = list(elasticsearch.helpers.scan(es, query={'query': {'match': {'user_id': user}}}, index='facebook', doc_type='post'))
        messages = [post['_source'].get('message') for post in posts if post['_source'].get('message') != '']
        #print(message)
        filtered_messages = filtering_url(messages)
        posts_dic[user] = filtered_messages
    return posts_dic

In [12]:
total_posts = get_posts(total_users)
male_posts_dict = get_posts(user_dict['male'])
female_posts_dict = get_posts(user_dict['female'])

# Using jieba as Chinese tokenizer, and calculating TFIDF

In [13]:
# Using jieba to tokenize user posts
def tokenize_posts(user_posts):
    seg_posts = {}
    for key, messages in user_posts.items():
        seg_posts[key] = [' '.join(jieba.cut(message, cut_all = False)) for message in messages]
    return seg_posts

In [14]:
male_seg_posts = tokenize_posts(male_posts_dict)
female_seg_posts = tokenize_posts(female_posts_dict)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.695 seconds.
Prefix dict has been built succesfully.


In [15]:
# Display tfidf score
def display_scores(vectorizer, tfidf_result):
    # http://stackoverflow.com/questions/16078015/
    scores = zip(vectorizer.get_feature_names(),
                 np.asarray(tfidf_result.sum(axis=0)).ravel())
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    index_scores = []
    for index, item in enumerate(sorted_scores):
        index_scores.append((index, item[1]))
    return sorted_scores

In [16]:
def calculate_tfidf(seg_posts):
    corpus = [segment for key, segments in seg_posts.items() for segment in segments]
    tfidf_vectorizer = TfidfVectorizer()
    tfidf = tfidf_vectorizer.fit_transform(corpus)
    tfidf_features = tfidf_vectorizer.get_feature_names()
    
    sorted_scores = display_scores(tfidf_vectorizer, tfidf)
    return sorted_scores

In [17]:
male_tfidf_scores = calculate_tfidf(male_seg_posts)
female_tfidf_scores = calculate_tfidf(female_seg_posts)

In [18]:
male_bag_of_words = [word[0] for word in male_tfidf_scores[:100]]
female_bag_of_words = [word[0] for word in female_tfidf_scores[:100]]

# Graph-based word categorization

In [19]:
%%time
word_index = {}
index = 1

#Building word index dictionary
for key, messages in total_posts.items():
    for message in messages:
        for word in list(message):
            if word not in word_index:
                word_index[word] = index
                index += 1

CPU times: user 544 ms, sys: 0 ns, total: 544 ms
Wall time: 555 ms


In [20]:
def building_weighted_graph(word_graph):
    weighted_word_graph = {}
    # Finding the maximun value of word frequency
    max_freq = max(word_graph.items(), key=operator.itemgetter(1))[1]

    for key, value in word_graph.items():
        weight = value / max_freq
        weighted_word_graph[key] = weight 
    return weighted_word_graph

In [21]:
def building_word_graph(user_posts):
    word_graph = {}
    for key, messages in user_posts.items():
        for message in messages:
            words = list(message)
            
            for i in range(len(words)-1):
                if (words[i], words[i+1]) not in word_graph:
                    word_graph[(words[i], words[i+1])] = 1
                else:
                    word_graph[(words[i], words[i+1])] += 1
    weighted_word_graph = building_weighted_graph(word_graph)
    return weighted_word_graph

In [22]:
male_weighted_graph = building_word_graph(male_posts_dict)
female_weighted_graph = building_word_graph(female_posts_dict)

In [23]:
import networkx as nx
import plotly.plotly as py
from plotly.graph_objs import *

In [24]:
G = nx.random_geometric_graph(200, 1)
# position is stored as node attribute data for random_geometric_graph
pos = nx.get_node_attributes(G, 'pos')

In [25]:
edge_trace = Scatter(
    x=[], 
    y=[], 
    line=Line(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

In [26]:
for key, messages in male_posts_dict.items():
    for message in messages:
        words = list(message)
        if '.' in words:
            print(message)
            #print(words)
            break

不要~~~很痛><....................標珩彤
Boo要這樣Brook Lopez QQ我還沒去看你打球...謝謝你陪籃網走過這些年儘管戰績不佳，歷經搬家也搭配過Vince Carter Delvin Harris Deron Williams到現在的Lin真的很喜歡你的投籃還有打板中距離以及上個球季練成的三分線去湖人加油籃網的1號位該由Lin還是DLO?雖然很喜歡林書豪但有那麼有天分的一號位或許第六人更適合林一大早就很不捨QQ我愛大羅Let's Go Nets!
多學多聞多問成功三要件·...........放屁啦，每次問個問題就崩潰我是要問什麼鬼學什麼毛啊⋯⋯
今年終於如願加入了活動組或許是種羨慕吧還記得當初問饅頭:妳會不會後悔找我加入活動組饅頭說:不會阿 這給我很大的鼓勵 很怕自己做不好其實謝謝柯柯 我知道你沒有我帥 可是你也不要氣餒 動組有你CARRY真的很棒 還記得有一天你在幫忙做火球 在那邊鬧脾氣 真的很好笑 很低能 辛苦了 上樑不正 這很難不去聯想就是說吼....謝謝饅頭 謝謝妳當初找我進動組 希望沒有讓妳後悔 動組有妳CARRY真的很讚 雖然小潘很變態 在動組大家的心目中妳根本是女神 真的很謝謝妳跟ㄎㄎ的包容與體貼 沒有你們 就沒有我們動組謝謝我的PARTNER馬小 真的很高興很跟你分在一組 太神拉 這是命運的安排 還記得一開始載你去發傳單 你就把手插進去我的口袋 其實還蠻害羞的哈哈 你真的太有才了 火球.賭場.大地各種CARRY 沒有你我都不知道該怎辦了 真得很感謝<3謝謝動組的夥伴 每天雖然充斥著髒話.變態的話 可是我很愛這種相處模式 大家一起忙一起瘋一起嘿嘿一起當農奴一起完成這次電資 大家真的超猛 舞.戲.大地.火線.賭場.RPG.夜教.主持等等.....真的不能不說我們是地表最強的活動組團隊  這次電資營讓我覺得最慶幸的是我認識了你們 有你們的的活動組才是活動組 謝謝你們讓我找回了自己最終也是最初的感動 以後一定要常常吃飯.聊天.內戰 愛你們謝謝2014電資營MISt strEEt全體工人 雖然過程中沒有盡善盡美 可是我們都努力過了 我們一起征服了 這樣就已經足夠了 YA有好多好多話想說 是我不夠體貼 總之辛苦了 嗯嗯哈哈嘻嘻嘿嘿亨亨嘖嘖蹦蹦QQㄜㄜ嗚嗚七槍七槍電資營讓我覺得最慶幸的是真的真的我認識了動組的你們
明年上班..

# Topic Modeling

In [100]:
with open('./extra_dictionary/stop_words.txt','r') as file:
    stopwords = [line.rstrip('\n') for line in file]

In [102]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [103]:
def stem_words(sentence):
    words = list(jieba.cut(sentence, cut_all = False))
    for word in words:
        if 'xd' in word or 'XD' in word or 'Xd' in word:
            index = words.index(word)
            words[index] = 'xd'
    stemmed_sentence = ' '.join([word for word in words if word not in stopwords])
    return stemmed_sentence

In [104]:
corpus = []
for key, messages in total_posts.items():
    for message in messages:
        stemmed_message = stem_words(message)
        corpus.append(stemmed_message)

In [105]:
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(corpus)

In [106]:
tfidf_sorted_scores = display_scores(tfidf_vectorizer, tfidf)

In [107]:
bag_of_words = [word[0] for word in tfidf_sorted_scores[:200]]

In [108]:
print(bag_of_words)

['xd', '真的', '今天', '可以', '大家', '一個', '我們', '還是', '覺得', '知道', '就是', '明天', '這樣', '哈哈', '看到', '不要', '不是', 'qq', '這麼', '一直', '時候', '一下', '好像', '感覺', '一起', '哈哈哈', '有人', '希望', '謝謝', '開始', '有點', '沒有', '還有', '現在', '雖然', '喜歡', '不過', '加油', '原來', '時間', '開心', '剛剛', '到底', '一樣', '快樂', '發現', '朋友', '如果', '一定', '台灣', '不會', '回家', '好吃', '很多', '分享', '不能', '但是', '好多', '應該', '晚安', '結果', '我要', '可是', '事情', '東西', '這是', '其實', '居然', '最近', '台北', '感謝', '正在', '已經', '一天', '只是', '一次', '根本', '世界', '第一次', '想要', '睡覺', '真是', '準備', '好久', '一點', '出來', '那麼', '突然', '人生', '只有', '晚餐', '需要', '回來', '天氣', '晚上', '這種', '生活', '好好', '老師', '胡牌', '想到', '他們', '終於', '生日', '照片', '參加', '問題', '畢業', '起來', '甚麼', '活動', '可能', '整個', 'you', '不想', '啊啊啊', '昨天', '結束', '地方', '完全', '繼續', '遇到', '不用', '一年', '是不是', '不到', '好奇', '只能', '兩個', '竟然', '今年', '心情', '努力', '早上', '果然', '每次', '第一', '工作', '看看', '大概', 'ya', '各位', '每天', '一堆', '期待', '一種', '可怕', '宿舍', '而且', '影片', '手機', '下雨', 'orz', '這次', '早餐', '早安', '厲害', '一些', '哈哈哈哈', '我覺', '以前', '小心', '非常', '電腦', '回到', '

In [88]:
tf_vectorizer = CountVectorizer()
tf = tf_vectorizer.fit_transform(corpus)

In [89]:
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [90]:
lda = LatentDirichletAllocation(n_topics=5, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [91]:
lda.fit(tf)

/usr/local/lib/python3.5/dist-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning:

n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21



LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=5, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [92]:
tf_feature_names = tf_vectorizer.get_feature_names()

In [93]:
print_top_words(lda, tf_feature_names, 15)

Topic #0:
哈哈哈 好吃 生日 正在 xd 手機 哈哈 可怕 下雨 快樂 哈哈哈哈 居然 tpa 到底 一句
Topic #1:
真的 我們 可以 今天 一個 還是 大家 xd 知道 就是 覺得 不是 看到 時候 這樣
Topic #2:
參加 活動 一起 神魔 快來 肚子 警察 桃園 解放 儒佳 幾天 中心 網址 快點 斷斷
Topic #3:
ahq 宿舍 qq xd 30 00 幹幹 鎖鎖鎖 __ 15 呵呵 day go we 14
Topic #4:
you and 圖靈機 it my 啊啊啊 網路 for 不起 me so be 我要 the 程式

